# Матчинг товаров с использованием библиотеки FAISS

Матчинг — это процесс поиска и сопоставления объектов внутри датасета (т. е. поиска match'ей). Он позволяет решать различные задачи — например, сворачивать одинаковые предложения в одну карточку товара на маркетплесах, строить систему рекомендаций и др.

В данном проекте мы будем искать матчи для товаров из запроса среди товаров, уже имеющихся в базе. В нашем распоряжении имеются следующие данные:

- `base.csv` — векторное представление товаров;
- `train.csv` — обучающая выборка: векторное представление товаров из запроса (query) и таргет, указывающий какой товар их базы является матчем;
- `test.csv` — тестовая выборка: векторное представление товаров из запроса, для которых нужно найти матч в base.


Для решения задачи матчинга мы применим векторный поиск с помощью библиотеки FAISS. FAISS — Facebook AI Research Similarity Search – это разработка команды Facebook AI Research для быстрого поиска ближайших соседей и кластеризации в векторном пространстве.

### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.preprocessing import StandardScaler

import faiss

plt.style.use('ggplot') # устанавливаем стиль графиков

## Загрузка данных

### base

Первым делом загрузим объемный датасет с информацией (в векторном виде) обо всех товарах.

In [2]:
base = pd.read_csv('base.csv')

In [3]:
base.shape

(2918139, 73)

И так, в нашем распоряжении данные о почти 3 млн товарах.

In [4]:
base.head()

,Id,0,1,2,3,4,5,6,7,8,...,62,63,64,65,66,67,68,69,70,71
0,0-base,-115.083890,11.152912,-64.426760,-118.88089,216.482440,-104.698060,-469.070588,44.348083,120.915344,...,-42.808693,38.800827,-151.76218,-74.389090,63.66634,-4.703861,92.933610,115.269190,-112.756640,-60.830353
1,1-base,-34.562202,13.332763,-69.787610,-166.53348,57.680607,-86.098370,-85.076666,-35.637436,119.718636,...,-117.767525,41.100000,-157.82940,-94.446806,68.20211,24.346846,179.937930,116.834000,-84.888941,-59.524610
2,2-base,-54.233746,6.379371,-29.210136,-133.41383,150.895830,-99.435326,52.554795,62.381706,128.951450,...,-76.397800,46.011803,-207.14442,127.325570,65.56618,66.325680,81.073490,116.594154,-1074.464888,-32.527206
3,3-base,-87.520130,4.037884,-87.803030,-185.06763,76.369540,-58.985165,-383.182845,-33.611237,122.031910,...,-70.647940,-6.358921,-147.20105,-37.692750,66.20289,-20.566910,137.206940,117.474100,-1074.464888,-72.915490
4,4-base,-72.743850,6.522049,43.671265,-140.60803,5.820023,-112.074080,-397.711282,45.182500,122.167180,...,-57.199104,56.642403,-159.35184,85.944724,66.76632,-2.505783,65.315285,135.051590,-1074.464888,0.319401


Товары "размещены" в 72 мерном пространстве — первый признак представляет собой `Id` товара.

In [5]:
base.sample(5)

,Id,0,1,2,3,4,5,6,7,8,...,62,63,64,65,66,67,68,69,70,71
109240,111829-base,-96.112480,3.427437,-56.177006,-141.14110,83.36199,-30.051083,-706.486506,17.786333,121.231980,...,-68.145004,1.519188,-186.16502,67.737878,68.413025,8.038726,108.629830,131.947240,-1074.464888,26.670467
2745892,4358424-base,-61.910070,16.940592,-37.620230,-124.25629,124.15181,-48.408062,-725.736157,6.579266,122.744460,...,-79.389960,52.715294,-135.81584,94.445635,69.286804,99.946900,98.581375,140.099490,-1074.464888,-14.797945
2071542,2978455-base,-78.380264,3.348803,-78.965870,-143.30316,152.41884,-76.038666,-120.758664,-24.036154,130.762590,...,-60.254640,22.152872,-237.98735,19.411931,69.198420,28.074724,108.128190,76.077420,-1074.464888,-58.070595
519943,578405-base,-94.860320,4.781488,-9.634884,-175.02700,93.40606,-117.012250,-267.583925,-42.746582,123.666725,...,-88.881680,24.992002,-149.59239,172.369056,66.220055,26.606184,234.950130,116.097565,-606.592929,49.655330
641663,730194-base,-115.114204,6.489541,-116.700290,-122.69424,203.01756,-50.847220,16.965378,-12.136667,131.480410,...,-56.835037,85.039444,-256.00604,97.519338,69.297710,61.772175,127.204710,76.542310,-278.994276,-84.856735


При взгляде на первые строки могло показаться, что индекс совпадает с числовым суффиксом товара, однако, выборка 5 случайных объектов позволяет определить, что это не так.

In [6]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2918139 entries, 0 to 2918138
Data columns (total 73 columns):
 #   Column  Dtype  
---  ------  -----  
 0   Id      object 
 1   0       float64
 2   1       float64
 3   2       float64
 4   3       float64
 5   4       float64
 6   5       float64
 7   6       float64
 8   7       float64
 9   8       float64
 10  9       float64
 11  10      float64
 12  11      float64
 13  12      float64
 14  13      float64
 15  14      float64
 16  15      float64
 17  16      float64
 18  17      float64
 19  18      float64
 20  19      float64
 21  20      float64
 22  21      float64
 23  22      float64
 24  23      float64
 25  24      float64
 26  25      float64
 27  26      float64
 28  27      float64
 29  28      float64
 30  29      float64
 31  30      float64
 32  31      float64
 33  32      float64
 34  33      float64
 35  34      float64
 36  35      float64
 37  36      float64
 38  37      float64
 39  38      float64
 40  

Все векторные признаки имеют тип данных `float64`, и в настоящий момент датасет занимает 1.6 Гб памяти.

In [7]:
base.describe()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
count,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,...,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06
mean,-8.622947e+01,8.080077e+00,-4.458080e+01,-1.466350e+02,1.113166e+02,-7.199138e+01,-3.922239e+02,2.035283e+01,1.236842e+02,1.244581e+02,...,-7.902286e+01,3.329735e+01,-1.547962e+02,1.415132e+01,6.779167e+01,2.354490e+01,7.495930e+01,1.155667e+02,-7.993390e+02,-4.779125e+01
std,2.489132e+01,4.953387e+00,3.863166e+01,1.984480e+01,4.634809e+01,2.818607e+01,2.716550e+02,6.421638e+01,6.356109e+00,6.443058e+01,...,3.045642e+01,2.888603e+01,4.122929e+01,9.895115e+01,1.823356e+00,5.534224e+01,6.134500e+01,2.117518e+01,3.854131e+02,4.174802e+01
min,-1.994687e+02,-1.391461e+01,-2.400734e+02,-2.326671e+02,-1.055830e+02,-2.110086e+02,-7.914699e+02,-3.018597e+02,9.315305e+01,-1.738719e+02,...,-2.205662e+02,-8.850774e+01,-3.539028e+02,-1.575944e+02,5.950944e+01,-2.331382e+02,-2.036016e+02,1.572448e+01,-1.297931e+03,-2.267801e+02
25%,-1.030654e+02,4.708491e+00,-6.955949e+01,-1.599051e+02,8.050795e+01,-9.137994e+01,-6.293318e+02,-2.222147e+01,1.194840e+02,8.176751e+01,...,-9.876390e+01,1.698862e+01,-1.807799e+02,-7.130038e+01,6.658096e+01,-1.251624e+01,3.377574e+01,1.016867e+02,-1.074465e+03,-7.566641e+01
50%,-8.623150e+01,8.038950e+00,-4.381661e+01,-1.467768e+02,1.118730e+02,-7.192230e+01,-4.222016e+02,2.080477e+01,1.238923e+02,1.234977e+02,...,-7.848812e+01,3.471502e+01,-1.539773e+02,1.382693e+01,6.781458e+01,2.341649e+01,7.492997e+01,1.160244e+02,-1.074465e+03,-4.859196e+01
75%,-6.925658e+01,1.147007e+01,-1.962527e+01,-1.333277e+02,1.423743e+02,-5.244111e+01,-1.566686e+02,6.391821e+01,1.279705e+02,1.672206e+02,...,-5.853355e+01,5.216429e+01,-1.273405e+02,9.966753e+01,6.902666e+01,5.975511e+01,1.158760e+02,1.295524e+02,-5.057445e+02,-1.971424e+01
max,2.151555e+01,2.993721e+01,1.609372e+02,-5.137478e+01,3.196645e+02,5.880624e+01,1.096325e+02,3.412282e+02,1.522612e+02,4.275421e+02,...,6.017411e+01,1.541678e+02,2.436099e+01,1.850981e+02,7.571203e+01,3.148988e+02,3.395738e+02,2.147063e+02,9.877081e+01,1.269732e+02


Отметим, что признаки не отнормированы.

Вызов метода .describe занял некоторое время из-за размера датасета — попробуем понизить его изменив типы данных и создав словарь для индексов товара.

In [8]:
base_dict = pd.DataFrame(base['Id'], index = base.index) # делаем новый датафрейм с индексом и ID
base_dict.to_csv("base_dict.csv", index=False)           # сохраним словарь в csv,
                                                         # в случае проблем с памятью переменную можно будет удалить
                                                         # а потом прочитать заново

Теперь можно удалить `Id` из основного датасета:

In [9]:
base = base.drop(columns=['Id'])

Небольшая потеря точности позволит сократить использование памяти в два раза.

In [10]:
base = base.astype('float32')

Отнормируем данные — большой разброс между значениями разных признаков сейчас затрудняет поиск реальных матчей.

In [11]:
scaler = StandardScaler()
base = scaler.fit_transform(base)

### train

In [12]:
train_data = pd.read_csv('train.csv')

In [13]:
train_data.head()

,Id,0,1,2,3,4,5,6,7,8,...,63,64,65,66,67,68,69,70,71,Target
0,0-query,-53.882748,17.971436,-42.117104,-183.93668,187.517490,-87.144930,-347.360606,38.307602,109.085560,...,70.107360,-155.80257,-101.965943,65.903790,34.457500,62.642094,134.763600,-415.750254,-25.958572,675816-base
1,1-query,-87.776370,6.806268,-32.054546,-177.26039,120.803330,-83.810590,-94.572749,-78.433090,124.915900,...,4.669178,-151.69771,-1.638704,68.170876,25.096191,89.974976,130.589630,-1035.092211,-51.276833,366656-base
2,2-query,-49.979565,3.841486,-116.118590,-180.40198,190.128430,-50.837620,26.943937,-30.447489,125.771164,...,78.039764,-169.14620,82.144186,66.008220,18.400496,212.409730,121.931470,-1074.464888,-22.547178,1447819-base
3,3-query,-47.810562,9.086598,-115.401695,-121.01136,94.652840,-109.255410,-775.150134,79.186520,124.003100,...,44.515266,-145.41675,93.990981,64.131350,106.061920,83.178760,118.277725,-1074.464888,-19.902788,1472602-base
4,4-query,-79.632126,14.442886,-58.903397,-147.05254,57.127068,-16.239529,-321.317964,45.984676,125.941284,...,45.028910,-196.09207,-117.626337,66.926220,42.456170,77.621765,92.479930,-1074.464888,-21.149351,717819-base


В трейне имеется номер запроса по товару (`Id`), его векторное предстваление в том же 72-мерном пространстве и информация о товаре-матче

Интересно будет посмотреть на матч для первого товара из запроса:

In [14]:
base_dict[base_dict['Id'] == '675816-base']

,Id
598613,675816-base


In [15]:
scaler.transform(train_data.drop(columns=['Id', 'Target']).head(1))

array([[ 1.29951853,  1.99688841,  0.06377414, -1.87967059,  1.64409957,
        -0.53762577,  0.1651479 ,  0.27959806, -2.29679437, -1.45962619,
         1.32070992,  0.33563286,  0.34833787, -0.074182  ,  1.46757774,
        -0.52832974, -0.13761893, -2.19922376, -0.52696653,  0.89398067,
         0.13583367, -0.89781577, -1.21825167,  0.63289176,  2.08869753,
         0.42159104, -0.13874557, -0.79563007, -2.00867326, -0.20007471,
        -0.53313854,  0.31707348,  0.68929787,  0.95999588,  0.31310802,
        -0.7932752 ,  1.57564291, -0.46032514,  1.08261035,  0.3847577 ,
        -0.3790614 , -0.244658  , -0.1444657 ,  0.62342473,  0.53313138,
         0.72798002, -0.48963424, -1.81578085, -0.91380803, -1.71248313,
         0.71461252, -0.33030673, -0.3105481 , -0.26974919,  0.24314398,
         1.49260542,  1.48756758, -0.39572116, -1.57330226,  1.64042511,
        -0.19343492, -0.17958294, -0.8669753 ,  1.27431895, -0.02440953,
        -1.17348095, -1.03538765,  0.19718403, -0.2

In [16]:
base[598613]

array([ 0.8988244 ,  1.8337636 ,  0.31869167, -1.1648953 ,  1.8890555 ,
       -0.1858217 ,  0.16514795, -0.11000545, -2.43123   , -1.2514124 ,
        0.89727324,  0.4314826 ,  0.11251274,  0.15258965,  2.3345332 ,
       -1.1134135 , -0.05690248, -2.4695697 , -1.4156506 , -0.08449763,
        0.3471339 ,  0.62431794, -1.2630662 ,  0.4742874 ,  2.4337287 ,
        0.42159107, -0.43362495, -0.50926805, -2.1432068 , -0.15034418,
       -0.575221  , -0.35922682,  0.72184   ,  1.5266184 , -0.20471278,
       -1.4419776 ,  1.2566878 , -0.34104428,  0.37731335,  0.54270613,
       -0.89426684, -0.18035163, -0.29362574,  0.16086134,  0.71248364,
        0.93715537, -0.3885359 , -1.2453146 , -0.5900144 , -1.9593219 ,
       -0.6220995 ,  0.00314887,  0.58806497, -0.40776905, -0.76953614,
        0.6052922 ,  0.93395084, -0.22825365, -1.7993007 ,  1.5215957 ,
       -0.04502272, -0.59186894, -0.927103  ,  1.6278056 ,  0.0729243 ,
        0.42745987, -0.49664134,  0.39475843, -0.5683024 ,  0.06

Видно, что некоторые часть координаты различаются, а другие (например, 70) — наоброт, почти полностью совпадают.

### test

In [17]:
test_data = pd.read_csv('test.csv')

In [18]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 73 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Id      100000 non-null  object 
 1   0       100000 non-null  float64
 2   1       100000 non-null  float64
 3   2       100000 non-null  float64
 4   3       100000 non-null  float64
 5   4       100000 non-null  float64
 6   5       100000 non-null  float64
 7   6       100000 non-null  float64
 8   7       100000 non-null  float64
 9   8       100000 non-null  float64
 10  9       100000 non-null  float64
 11  10      100000 non-null  float64
 12  11      100000 non-null  float64
 13  12      100000 non-null  float64
 14  13      100000 non-null  float64
 15  14      100000 non-null  float64
 16  15      100000 non-null  float64
 17  16      100000 non-null  float64
 18  17      100000 non-null  float64
 19  18      100000 non-null  float64
 20  19      100000 non-null  float64
 21  20      100

In [19]:
test_data.sample(5)

,Id,0,1,2,3,4,5,6,7,8,...,62,63,64,65,66,67,68,69,70,71
45373,145373-query,-45.475197,2.815635,-26.068275,-136.51530,114.54130,-63.615070,-699.332470,-113.782585,124.507740,...,-100.206474,51.686270,-122.381996,-140.950650,71.173850,35.158813,67.881950,130.823900,-1279.178394,-68.231450
89178,189178-query,-65.872086,8.739495,-34.580215,-134.65695,141.78589,-121.425514,-503.696200,-136.588030,127.123245,...,-56.872450,19.983128,-157.493560,52.520428,66.498030,-89.243620,-104.524070,74.410680,-1117.043459,-131.664780
75182,175182-query,-83.645320,4.899579,41.789260,-156.07639,116.35464,-73.284680,-526.181780,79.821570,130.357970,...,-96.193610,19.539772,-114.803410,55.899709,66.863914,57.164220,-53.600777,100.541230,13.265891,-56.410416
22525,122525-query,-112.298935,10.793612,3.628201,-140.30923,187.62802,-67.639824,-97.550520,-63.034386,128.966260,...,-33.770910,51.394890,-167.621290,108.571094,68.740330,90.557710,87.694530,146.667970,-595.683884,-64.662810
66143,166143-query,-58.698982,7.153547,-70.833750,-141.98482,148.40082,-60.932007,-759.626065,-15.901358,123.264850,...,-99.438210,52.035583,-138.414500,171.300829,69.392410,73.554790,-16.920624,84.248764,-427.467740,-48.847450


То же 72-х мерное пространство, которое следует отнормировать перед использованием.

## Использование FAISS

Для поиска матчей воспользуемся Inverted File index из FAISS. Количество кластеров определим как квадратный корень из `base`.


In [20]:
n_clusters = np.sqrt(len(base)).round()

In [21]:
n_clusters

1708.0

In [22]:
dim = 72  # размерность пространства
k = 1708  # количество центров
quantiser = faiss.IndexFlatL2(dim) 
index = faiss.IndexIVFFlat(quantiser, dim, k)

In [23]:
index = faiss.index_factory(dim, 'IVF1708,Flat')
index.train(base)

print(index.is_trained)  
print(index.ntotal)   
index.add(base)
print(index.ntotal)

True
0
2918139


In [24]:
topn = 10 # количество ближайших товаров-векторов, которые могут оказаться матчем

In [25]:
test_dict = pd.DataFrame(test_data['Id'], index = test_data.index) # делаем новый датафрейм с индексом и ID
test_dict.to_csv("test_dict.csv", index=False)  

In [26]:
test_data = test_data.drop(columns=['Id']).astype('float32')

In [27]:
test_data = scaler.transform(test_data)

In [28]:
index.nprobe = 100  # Проходим по топ-100 центроид для поиска top-n ближайших соседей
D, I = index.search(test_data, topn)

In [29]:
test_data.shape

(100000, 72)

In [30]:
I.shape

(100000, 10)

И так, мы нашли ближайшие вектора-товары для каждого товара из тестового запроса — время проверить работу FAISS'a путем отправки сабмита

In [31]:
id_base_dict = dict(base_dict['Id'])

predicted_list = []
for candidates in I:
    predicted_list.append(' '.join([id_base_dict[candidate] for candidate in candidates]))
    
#формируем ответ
answer = test_dict[['Id']]
answer['Predicted'] = predicted_list

answer.to_csv('submission_scaled.csv', index=False)

### Сatboost ranking

Текущий сабмит дает метрику recall@10 0.69 на паблике, и дальшейшее улучшение работы связано с использованием тренировочных данных для переранжирования ближайших векторов, найженных с помощью Faiss.

Далее мы попробуем два подхода для переранжировки найденных соседей:

- Для улучшения работы модели мы добавим в тренировочные данные признаки-координаты матчей, и поставим им класс 1 - т.е. матч. Затем создадим ложные выборки из случайных объектов. Учтем дисбаланс классов — отричательныйх объектов-нематчей нужно создать сильно больше. При поиске faiss'ом будем искать не 10 ближайших векторов, а 50. Для всех 50 кандидатов будем смотреть вероятность принадлежности к классу 1 и выбирать наиболее вероятные объекты-вектора.

- используем готовый класс CatBoostRanker для переранжирования.

<div class="alert alert-info"> <b>ℹ️ Комментарий: </b> Дальнейшую часть работы по переранжированию выполнить не успел, т.к. был вынужден работать в выходные - оставьте доступ к late submussion, пожалуйста </div>

In [32]:
### очистим память удалив лишние объекты и прочтем необходимые исходники заново.

In [33]:
from catboost import CatBoostRanker, Pool, MetricVisualizer
from copy import deepcopy
import os